In [8]:
from models.Blocks import DiscriminatorTop
import torch
import torch.nn as nn
from models.CustomLayers import *
import numpy as np

In [2]:
def nf(stage):
    return min(int(8192 / (2.0 ** (stage * 1.0))), 512)

In [3]:
act, gain = {'relu': (torch.relu, np.sqrt(2)),'lrelu': (nn.LeakyReLU(negative_slope=0.2), np.sqrt(2))}['lrelu']


In [4]:
layer = DiscriminatorTop(4, 1,in_channels=nf(2), intermediate_channels=nf(2),gain=gain, use_wscale=True, activation_layer=act)
        

In [7]:
mbstd_group_size = 4
mbstd_num_features = 1
in_channels = nf(2)
intermediate_channels  = nf(2)
gain = gain
use_wscale = True
activation_layer = act
resolution=4
in_channels2=None
output_features=1
last_gain=1

In [27]:
layers = []
if mbstd_group_size > 1:
    layers.append(('stddev_layer', StddevLayer(mbstd_group_size, mbstd_num_features)))

if in_channels2 is None:
    in_channels2 = in_channels

layers.append(('conv', EqualizedConv2d(in_channels + mbstd_num_features, in_channels2, kernel_size=3,
                                        gain=gain, use_wscale=use_wscale)))
layers.append(('act0', activation_layer))
layers.append(('view', View(-1)))
layers.append(('dense0', EqualizedLinear(in_channels2 * resolution, intermediate_channels,
                                            gain=gain, use_wscale=use_wscale)))
layers.append(('act1', activation_layer))
layers.append(('dense1', EqualizedLinear(intermediate_channels, output_features,gain=last_gain, use_wscale=use_wscale)))


In [20]:
a = torch.rand([128, 512, 2, 2])

In [21]:
out1 = layers[0][1](a)

In [22]:
out2 = layers[1][1](out1)

In [23]:
out3 = layers[2][1](out2)

In [24]:
out4 = layers[3][1](out3)

In [30]:
out5 = layers[4][1](out4)

here
torch.Size([512, 2048])


In [32]:
out6 = layers[5][1](out5)

In [33]:
out7 = layers[6][1](out6)

here
torch.Size([1, 512])


In [35]:
F.linear(torch.rand([512, 2048]),torch.rand([512, 2048]),torch.rand([512]))

tensor([[503.1995, 495.2905, 512.1022,  ..., 494.5173, 505.1496, 507.6604],
        [510.3517, 505.0934, 512.6149,  ..., 513.4971, 511.3862, 518.2028],
        [505.2905, 500.9744, 511.7928,  ..., 513.3304, 502.2877, 519.8686],
        ...,
        [510.4067, 508.1538, 521.6430,  ..., 518.9274, 513.8808, 523.3340],
        [514.3096, 511.9636, 530.2161,  ..., 520.1252, 522.5251, 537.9234],
        [499.7108, 500.6949, 513.5479,  ..., 508.0507, 497.5432, 511.3209]])